## Assignment 3:  Convolutional networks for TF ChIP-seq data

In this assignment you will build on the convolutional networks we looked at in class and work on ChIP-seq data for four transcription factors in arabidopsis.


### Part 1: Data Preparation

In this assignment you will work with ChIP-seq for three arabidopsis transcription factors:  AGL16, GRF1, and AMS.  The peaks that represent their binding sites in the arabidopsis genome are available in the following links:

* AGL16 ([bed file](https://biobigdata.nju.edu.cn/ChIPHub_download/arabidopsis_thaliana/SRP187795/hammock/AGL16.target.all.bed.gz))
* GRF1 ([bed file](https://biobigdata.nju.edu.cn/ChIPHub_download/arabidopsis_thaliana/SRP002566/hammock/SRX021610.peak.all.bed.gz))
* AMS ([bed file](https://biobigdata.nju.edu.cn/ChIPHub_download/arabidopsis_thaliana/SRP188198/hammock/SRX5507861.peak.all.bed.gz))

These files are in [bed format](https://en.wikipedia.org/wiki/BED_(file_format)), and contain the information on the genomic locations where the ChIP-seq peaks have been detected.  The linked wikipedia article provides the information you need about the format of these files.  Your task is to extract sequences of length 500 centered at the location of each peak, which you will provide as input to the convolutional network you train.  

In order to extract the sequences associated with the peaks you will need the genomic sequence for arabidopsis.  This is available from the [Ensembl plants arabidopsis portal](https://plants.ensembl.org/Arabidopsis_thaliana/Info/Index).  In that page click on "Download DNA sequence (FASTA)", and the first five files provide the sequences for the five arabidopsis chromosomes.

For reference, we computed the sequences associated with AGL16 peaks (link is in the assignment page in Canvas).

Your final data preparation task is to prepare a labeled dataset with positive examples that correspond to the peak sequences.  As negative examples, use random permutations of the positive examples.  Create one permutation from each positive example.  How many examples did you obtain for each transcription factor?

In [1]:
# Task
# - Load data
# given bed file -> convert to fasta file (example database\AGL16_peak_seq.fasta)
# - Generate dataset
# given a fasta file -> a dataset with negative example made from postive

In [2]:
# lets look at the bed files 
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

positions = {}
with open('database\AGL16.target.all.bed') as f:
    for line in f:
        chr, start, stop, name, score, strand, thickStart, thickEnd, itemRgb, blockcount, blockSize, blockStart, _, _ = line.split()
        # idk what the last two are there is only supposed to be 12 according to wikipedia
        positions[name] = (chr, int(start), int(stop), (int(start)+int(stop))//2)

genome_locations = [r'database\Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa',
                    r'database\Arabidopsis_thaliana.TAIR10.dna.chromosome.2.fa',
                    r'database\Arabidopsis_thaliana.TAIR10.dna.chromosome.3.fa',
                    r'database\Arabidopsis_thaliana.TAIR10.dna.chromosome.4.fa',
                    r'database\Arabidopsis_thaliana.TAIR10.dna.chromosome.5.fa']
genome = {}
for l in genome_locations:
    genome = genome | SeqIO.to_dict(SeqIO.parse(open(l), 'fasta'), lambda rec: 'Chr'+rec.name)

# Extract 500bp sequences centered at midpoints
sequences = []
for name, (chr, start, stop, mid_point) in positions.items():
    # Calculate the start and end positions for a 500bp sequence centered at midpoint
    seq_start = max(0, mid_point - 250)  # Ensure we don't go below 0
    seq_end = min(len(genome[chr].seq), mid_point + 250)  # Ensure we don't go beyond chromosome length
    
    # Extract the sequence
    sequence = genome[chr].seq[seq_start:seq_end]
    
    # Create a SeqRecord
    record = SeqRecord(sequence, id=name, description=f"Chromosome {chr}, midpoint {mid_point}")
    sequences.append(record)

# Write to FASTA file
SeqIO.write(sequences, "AGL16_peak_seq.fasta", "fasta")
print(f"Extracted {len(sequences)} sequences and saved to FASTA file.")




<>:7: SyntaxWarning: invalid escape sequence '\A'
<>:7: SyntaxWarning: invalid escape sequence '\A'
c:\temp\ipykernel_22200\2856302313.py:7: SyntaxWarning: invalid escape sequence '\A'
  with open('database\AGL16.target.all.bed') as f:


Extracted 4328 sequences and saved to FASTA file.


In [3]:
import os

def get_sequences(genome_paths = [r'database\Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa',
                                                                r'database\Arabidopsis_thaliana.TAIR10.dna.chromosome.2.fa',
                                                                r'database\Arabidopsis_thaliana.TAIR10.dna.chromosome.3.fa',
                                                                r'database\Arabidopsis_thaliana.TAIR10.dna.chromosome.4.fa',
                                                                r'database\Arabidopsis_thaliana.TAIR10.dna.chromosome.5.fa'], 
                                        bed_path='database\AGL16.target.all.bed',
                                        len_of_seqs=500):
    
    positions = {}
    with open(bed_path) as f:
        for line in f:
            chr, start, stop, name, score, strand, thickStart, thickEnd, itemRgb, blockcount, blockSize, blockStart, _, _ = line.split()
            # idk what the last two are there is only supposed to be 12 according to wikipedia
            positions[name] = (chr, int(start), int(stop), (int(start)+int(stop))//2)

    genome = {}
    for l in genome_paths:
        genome = genome | SeqIO.to_dict(SeqIO.parse(open(l), 'fasta'), lambda rec: 'Chr'+rec.name)

    # Extract 500bp sequences centered at midpoints
    sequences = []
    for name, (chr, start, stop, mid_point) in positions.items():
        # Calculate the start and end positions for a 500bp sequence centered at midpoint
        seq_start = max(0, mid_point - len_of_seqs//2)  # Ensure we don't go below 0
        seq_end = min(len(genome[chr].seq), mid_point + len_of_seqs//2)  # Ensure we don't go beyond chromosome length
        
        # Extract the sequence
        sequence = genome[chr].seq[seq_start:seq_end]
        
        # Create a SeqRecord
        record = SeqRecord(sequence, id=name, description=f"Chromosome {chr}, midpoint {mid_point}")
        sequences.append(record)

    # Write to FASTA file
    SeqIO.write(sequences, os.path.basename(bed_path)+".fasta", "fasta")
    print(f"Extracted {len(sequences)} sequences and saved to FASTA file.")
    return sequences

<>:8: SyntaxWarning: invalid escape sequence '\A'
<>:8: SyntaxWarning: invalid escape sequence '\A'
c:\temp\ipykernel_22200\1604433860.py:8: SyntaxWarning: invalid escape sequence '\A'
  bed_path='database\AGL16.target.all.bed',


In [4]:
import numpy as np
nucleotide_indexes = {'a':0, 't':1, 'g':2, 'c':3, 'w': (0,1), 'n': (0,1,2,3)}

def one_hot_encode(seq):
    oneHotEncode = np.zeros([4,len(seq)])
    for i, n in enumerate(seq):
        oneHotEncode[nucleotide_indexes[n.lower()], i] = 1
    return oneHotEncode

def generate_dataset(sequences):
    positive_seqs = []
    negative_seqs = []

    for seq_record in sequences:
        positive_seqs.append(one_hot_encode(str(seq_record.seq)))
        
        seq_list = list(str(seq_record.seq))
        np.random.shuffle(seq_list)
        permuted_seq = ''.join(seq_list)
        negative_seqs.append(one_hot_encode(permuted_seq))

    positive_labels = np.ones(len(positive_seqs))
    negative_labels = np.zeros(len(negative_seqs))

    X = positive_seqs + negative_seqs
    y = np.concatenate([positive_labels, negative_labels])

    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = [X[i] for i in indices]
    y = y[indices]



    print(f"Dataset created with {len(positive_seqs)} positive examples and {len(negative_seqs)} negative examples")
    return X, y

In [5]:
X, y = generate_dataset(get_sequences())

Extracted 4328 sequences and saved to FASTA file.
Dataset created with 4328 positive examples and 4328 negative examples


### Part 2:  

As discussed in class, deeper networks with multiple layers of convolution *can* improve a network's performance.  Your task here is to extend the implementation provided in class to have three layers of convolution.  In addition, implement early stopping based on performance on the validation set (essentially, continue training until the validation loss stops decreasing).
Finally, train each network two or three times, and choose the best performing network based on the performance on the validation set as the one to evaluate on the test set.
In your experiments, set aside 20% of the data for testing, 20% for validation, and 60% for training.
Compare the accuracy of your network to that of a one layer CNN.  Accuracy should be measured using the area under the ROC curve.  In the next part of the assignment you will get to tune its parameters to try and improve its performance.
Note that for all datasets you should be able to obtain accuracy of around 0.9 or better.

In [ ]:
import torch
from torch import nn
import sklearn as sk

model = nn.Sequential(nn.Conv2d(in_channels=4, out_channels=3, kernel_size=(3, 1), stride=1),
                      nn.ReLU(),
                      nn.Conv2d(in_channels=4, out_channels=3, kernel_size=(3, 1), stride=1),
                      nn.ReLU(),
                      nn.Conv2d(in_channels=4, out_channels=3, kernel_size=(3, 1), stride=1))

### Part 3:  experiments with network architecture and hyperparameters


With the implementation you created in Part 2, your next task is to explore the space of hyperparameters and architecture choices to determine their effect on the performance of your three-layer network.  Choose three aspects of the network to explore (e.g. the learning rate, whether dropout is helpful, the choice of activation function, etc.).  Discuss your results.  Which aspects of the model seem to have the most effect on the accuracy of the network?  Do the best parameter values vary from dataset to dataset?  Is your three layer network able to match or exceed the performance of a single layer network?  Hint:  it should!

### Coding and reporting your results

In your notebook, I do not want to see repetitive code.  Such code belongs in a function!
In your reporting, make sure your results are clearly presented.  I recommend using a table format, and your table can be populated by your code.  pandas DataFrame objects render nicely in Jupyter notebooks.  Here's an example:


In [ ]:
import pandas as pd
import numpy as np

data = [
    ['AGL16', 'Dropout', 0.2, 0.92 ],
    ['GRF1', 'Dropout', 0.3, 0.9],
]
pd.DataFrame(data, columns = ['Dataset', 'Hyperparameter', 'Value', 'Accuracy'])

#### Grading

```
Part 1: dataset creation (20 pts)
Part 2: implementation of three layer network, early stopping, and multiple training (40 pts)
Part 3: experiments on network architecture (40 pts)
```